# Parsing SEC Filing XBRL Document


## Objective

Parse the filing XBRL file to create a DOM like structure that represent the filing data

## References

* [XBRL Specification - Extensible Business Reporting Language (XBRL) 2.1](https://www.xbrl.org/Specification/XBRL-2.1/REC-2003-12-31/XBRL-2.1-REC-2003-12-31+corrected-errata-2013-02-20.html)

* [List of US GAAP Standards](https://xbrlsite.azurewebsites.net/2019/Prototype/references/us-gaap/)
* [XBRL US - List of Elements](https://xbrl.us/data-rule/dqc_0015-le/)

**Element Version**|**Element ID**|**Namespace**|**Element Label**|**Element Name**|**Balance Type**|**Definition**
:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:
1|1367|us-gaap|Interest Expense|InterestExpense|debit|Amount of the cost of borrowed funds accounted for as interest expense.
2|2692|us-gaap|Cash and Cash Equivalents, at Carrying Value|CashAndCashEquivalentsAtCarryingValue|debit|Amount of currency on hand as well as demand deposits with banks or financial institutions. Includes other kinds of accounts that have the general characteristics of demand deposits. Also includes short-term, highly liquid investments that are both readily convertible to known amounts of cash and so near their maturity that they present insignificant risk of changes in value because of changes in interest rates. Excludes cash and cash equivalents within disposal group and discontinued operation.

## XBRL Element

* [Understanding the Financial Report Logical System](https://www.youtube.com/playlist?list=PLqMZRUzQ64B7EWamzDP-WaYbS_W0RL9nt)
* [XBRL - What is us-gaap:OperatingSegmentsMember element anb where is it defined?](https://money.stackexchange.com/questions/148010/xbrl-what-is-us-gaapoperatingsegmentsmember-element-anb-where-is-it-defined)

### Example
For instance, Qorvo 2020 10K

* [XBRL/rfmd-20210403_htm.xml](https://www.sec.gov/Archives/edgar/data/1604778/000160477821000032/rfmd-20210403_htm.xml)
* [HTML/rfmd-20210403.htm)](https://www.sec.gov/Archives/edgar/data/1604778/000160477821000032/rfmd-20210403.htm):

```
<us-gaap:cashandcashequivalentsatcarryingvalue contextref="*" decimals="-3" id="..." unitref="usd">
  1397880000
</us-gaap:cashandcashequivalentsatcarryingvalue>,
<us-gaap:cashandcashequivalentsatcarryingvalue contextref="***" decimals="-3" id="..." unitref="usd">
  714939000
</us-gaap:cashandcashequivalentsatcarryingvalue>,
<us-gaap:cashandcashequivalentsatcarryingvalue contextref="***" decimals="-3" id="..." unitref="usd">
 711035000
</us-gaap:cashandcashequivalentsatcarryingvalue>
```

Corresponds to the Cash and Cash equivalents in the Cash Flow statement.

<img src="../image/edgar_qorvo_2020_10K_CF.png" align="left" width=800 />

---
# Setup

In [1]:
from typing import (
    List,
    Dict
)
import datetime
import dateutil
import calendar
import re
import requests
import unicodedata
import bs4
from bs4 import BeautifulSoup
from IPython.core.display import (
    display, 
    HTML
)

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: ('%f' % x).rstrip('0').rstrip('.'))
pd.set_option('display.colheader_justify', 'center')

In [2]:
%%html
<style>
table {float:left}
</style>

In [3]:
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Global Constant

In [4]:
# regexp to extract numeric string
REGEXP_NUMERIC = re.compile(r"\s*[\d.-]*\s*")

re.match(REGEXP_NUMERIC, " -123.12 ")

<re.Match object; span=(0, 9), match=' -123.12 '>

In [5]:
FS_TYPE_10K = "10-K"
FS_TYPE_10Q = "10-Q"

---
# Load EDGAR Filing XBRL

Download the ```_htm.xml``` file from EDGAR. SEC now requires user-agent header.

In [6]:
# QORVO 2021 10K
CIK = '1604778'
ACCESSION = '000160477821000032'
FS_TYPE = FS_TYPE_10K

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "rfmd-20210403_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/1604778/000160477821000032/rfmd-20210403_htm.xml'

In [7]:
# define the url to specific html_text file
CIK = '0001000230'
ACCESSION = '000143774921028951'
FS_TYPE = FS_TYPE_10K

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "occ20211031_10k_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/0001000230/000143774921028951/occ20211031_10k_htm.xml'

In [8]:
# AMKOR 2021 10K
CIK = '1047127'
ACCESSION = '000104712721000043'
FS_TYPE = FS_TYPE_10Q

FILING_DIR_URL = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{ACCESSION}"
XBRL_NAME = "amkr-20210930_htm.xml"
XBRL_URL = "/".join([FILING_DIR_URL, XBRL_NAME])

XBRL_URL

'https://www.sec.gov/Archives/edgar/data/1047127/000104712721000043/amkr-20210930_htm.xml'

In [9]:
headers = {"User-Agent": "Company Name myname@company.com"}
response = requests.get(XBRL_URL, headers=headers)

if response.status_code == 200:
    content = response.content.decode("utf-8") 
else:
    assert False, f"{XBRL_URL} failed with status {response.status_code}"

In [10]:
soup = BeautifulSoup(content, 'html.parser')

## Repoting period

Each 10-K and 10-Q XBRL has the reporting period for the filing. To exclude the other period, e.g. pervious year or quarter, use the ```context id``` for the reporting period. **Most** 10-K, 10-Q specify the annual period with the first ```<startDate> and <endDate>``` tags.

For instances:

### QRVO 10-K 2020

```
<context id="ifb6ce67cf6954ebf88471dd82daa9247_D20200329-20210403">
    <entity>
    <identifier scheme="http://www.sec.gov/CIK">0001604778</identifier>
    </entity>
    <period>
        <startDate>2020-03-29</startDate>
        <endDate>2021-04-03</endDate>
    </period>
</context>
```

### AMKR 10-K 2020

```
<context id="i5fac0a392353427b8266f185495754d3_D20200101-20201231">
    <entity>
    <identifier scheme="http://www.sec.gov/CIK">0001047127</identifier>
    </entity>
    <period>
        <startDate>2020-01-01</startDate>
        <endDate>2020-12-31</endDate>
    </period>
</context>
```

### AAPL 10-Q 4th QTR 2020

```
<context id="i6e431846933d461fb8c8c0bdf98c9758_D20200927-20201226">
    <entity>
    <identifier scheme="http://www.sec.gov/CIK">0000320193</identifier>
    </entity>
    <period>
        <startDate>2020-09-27</startDate>
        <endDate>2020-12-26</endDate>
    </period>
</context>
```

**However, there are companies that do not have this manner**. For instance [10-K for OPTICAL CABLE CORPORATION(CIK=0001000230)](https://www.sec.gov/Archives/edgar/data/1000230/000143774921028951/occ20211031_10k_htm.xml) has the same start and end dates at first.

```
<context id="d202110K">
	<entity>
		<identifier scheme="http://www.sec.gov/CIK">0001000230</identifier>
	</entity>
	<period>
		<startDate>2021-10-31</startDate>   # <-----
		<endDate>2021-10-31</endDate>
	</period>
</context>
<context id="d_2020-11-01_2021-10-31">
	<entity>
		<identifier scheme="http://www.sec.gov/CIK">0001000230</identifier>
	</entity>
	<period>
		<startDate>2020-11-01</startDate>   # <-----
		<endDate>2021-10-31</endDate>
	</period>
</context>
```

The report uses the 2nd for 2021 F/S element value but does not use the first one.

**B/S**
```
<us-gaap:Assets 
    contextRef="i_2021-10-31"    # <-----
    decimals="INF" 
    id="c79893606" 
    unitRef="USD"
>
  37916530
</us-gaap:Assets>
```

**P/L**
```
<us-gaap:RevenueFromContractWithCustomerIncludingAssessedTax 
  contextRef="d_2020-11-01_2021-10-31"     # <-----
  decimals="INF" 
  id="c79893662" 
  unitRef="USD"
>
  59136294
</us-gaap:RevenueFromContractWithCustomerIncludingAssessedTax>
```

<img src='../image/edgar_optical_cable_2021_10K.png' align="left" width=500/>

### Get the period from the 1st

For now, just get the period from the 1st **period** element.

In [11]:
CONTEXT = None
for context in soup.find_all('context'):
    if context.find('period') and context.find('period').find('enddate'):
        CONTEXT = context
        break        
        
assert CONTEXT is not None, "Report period not found"

cik_value_in_statement = int(CONTEXT.find('identifier').text.strip())
assert int(CIK) == cik_value_in_statement, \
    "The CIK %s in statement does not match %s" % (cik_value_in_statement, int(CIK))

CONTEXT_ID = CONTEXT['id']
PERIOD = CONTEXT.find('period').find('enddate').text.strip()

print(f"PERIOD is {PERIOD}. CONTEXT_ID is {CONTEXT_ID}")

PERIOD is 2021-09-30. CONTEXT_ID is i9289f6e06c5a47e09383379425d29391_D20210101-20210930


### Regexp to find all the contexts that match with PERIOD

10-K, 10-Q F/S uses multiple contexts to refer to the F/S element values for the **period**. Collect all the contexts for the **period**.

In [12]:
def get_target_contexts(soup, period, type=FS_TYPE_10K):
    target_contexts = []

    end_date = dateutil.parser.parse(period)
    duration = datetime.timedelta(days=365) if type == FS_TYPE_10K else datetime.timedelta(days=90)

    # The StartDate of the valid context is within +/- 30 days of the starting date of the period.
    # If the report is 10-K and the period end is 2021-10-31, then the validate date is between
    # 2020-10-1 and 2020-11-1
    from_date = end_date - duration - datetime.timedelta(days=30)
    to_date = end_date - duration + datetime.timedelta(days=30)

    # Contexts with startdate/enddate
    for context in soup.find_all('context'):
        # print("-"*100)
        # print(context)
        if context.find('period') and context.find('period').find('enddate', string=period):
            start = dateutil.parser.parse(context.find('period').find('startdate').text)
            if start == end_date or (from_date < start and start < to_date):
                target_contexts.append(context['id'])

    
    # Contexts with 'instant' element
    target_contexts.extend([
        context['id'] for context in soup.find_all('context') 
        if context.find('period') and context.find('period').find(['instant'], string=PERIOD)
    ])
    
    return target_contexts

In [13]:
contexts = get_target_contexts(soup=soup, period=PERIOD, type=FS_TYPE)
contexts_regexp = "|".join(contexts)
CONTEXT_REGEXP = re.compile(contexts_regexp)

# Constant

In [14]:
# XBRL Namespace
NAMESPACE = "us-gaap"

# XBRL attribute conditions to match when extracting FS elements
ATTRIBUTES = {
    "contextref": CONTEXT_REGEXP,
    "decimals": True, 
    "unitref": True
}

In [15]:
CREDIT_ITEMS = set(map(str.lower, [
    f"{NAMESPACE}:Revenues",
    f"{NAMESPACE}:GrossProfit",
]))

DEBIT_ITEMS = set(map(str.lower, [
    f"{NAMESPACE}:CostOfRevenue",
    f"{NAMESPACE}:CostOfGoods",
    f"{NAMESPACE}:CostOfGoodsAndServicesSold",
    f"{NAMESPACE}:OperatingExpenses",
    f"{NAMESPACE}:ResearchAndDevelopmentExpense",
    f"{NAMESPACE}:SellingGeneralAndAdministrativeExpense",
    f"{NAMESPACE}:OtherCostAndExpenseOperating",
    f"{NAMESPACE}:IncomeTaxExpenseBenefit",
    f"{NAMESPACE}:InterestExpense",
]))

# Utilities

In [16]:
def assert_bf4_tag(element):
    assert isinstance(element, bs4.element.Tag), \
    f"Expected BS4 tag but {element} of type {type(element)}"

In [17]:
def get_element_hash(element):
    """Generate the financial element hash key to uniquely identify an financial element record
    In a F/S, the same element, e.g. gaap:CashAndCashEquivalentsAtCarryingValue can be used at
    multiple places, one in B/S and one in P/L.
    
    To be able to identify if two elements are the same, provides a way to be able to compare
    two elements by generating a hash key from the attributes of an element.

    Use name, unitref, decimals, text to generate the has.
    
    Args:
        element: bs4.element.Tag for an financial element
    Returns: hash key
    """
    assert isinstance(element, bs4.element.Tag)
    # key = f"{element.name}{element['unitref']}{element['contextref']}{element.text}"
    key = f"{element.name}{element['unitref']}{element.text}"
    return hash(key)

In [18]:
def find_financial_elements(soup, element_names, attributes=ATTRIBUTES):
    """Find the financial statement elements from the XML/HTML source.
    Args:
        soup: BS4 source
        element_names: String or regexp instance to select the financial elements.
        attribute: tag attributes to select the financial elements
    Returns:
        List of BS4 tag objects that matched the element_names and attributes.
    """
    assert isinstance(soup, BeautifulSoup)
    assert isinstance(element_names, re.Pattern) or isinstance(element_names, str)

    
    names = element_names.lower() if isinstance(element_names, str) else element_names

    elements = soup.find_all(
        name=names,
        string=REGEXP_NUMERIC,
        attrs=attributes
    )
    
    # Select unique elements
    hashes = set([]) 
    results = []
    if elements is not None and len(elements) > 0:
        for element in elements:
            hash_value = get_element_hash(element)
            if hash_value not in hashes:
                results.append(element) 
                hashes.add(hash_value)

    return results

In [19]:
def get_financial_element_numeric_values(elements):
    # assert isinstance(elements, bs4.element.ResultSet) or isinstance(elements[0], bs4.element.Tag)
    assert_bf4_tag(elements[0])
    
    values = []
    for element in elements:
        assert re.match(REGEXP_NUMERIC, element.text.strip()), f"Element must be numeric but {element.text}"
        values.append(float(element.text))
        
    return values

In [20]:
def get_records_for_financial_elements(elements):
    # assert isinstance(elements, bs4.element.ResultSet) or isinstance(elements[0], bs4.element.Tag)
    assert_bf4_tag(elements[0])
    
    return [
        [
            'debit' if element.name in DEBIT_ITEMS else 'credit',
            element.name, 
            element['unitref'], 
            int(element['decimals']) if element['decimals'].lower() != 'inf' else np.inf, 
            float(element.text), 
            element['contextref']
        ]
        for element in elements
    ]

In [21]:
def get_records_for_financial_element_names(soup, names: List[str]):
    elements = find_financial_elements(soup=soup, element_names=names)
    if len(elements) > 0:
        display_elements(elements)
        return get_records_for_financial_elements(elements)
    else:
        return []

In [22]:
def get_values_for_financial_element_names(soup, names: List[str]):
    elements = find_financial_elements(soup=soup, element_names=names)
    if len(elements) > 0:
        display_elements(elements)
        return get_financial_element_numeric_values(elements)
    else:
        return []

In [23]:
def display_elements(elements):
    assert isinstance(elements, bs4.element.ResultSet) or isinstance(elements[0], bs4.element.Tag)
    for element in elements: # decimals="-3" means the displayed value is divied by 1000.
        print(f"{element.name:80} {element['unitref']:5} {element['decimals']:5} {element.text:15}")

In [24]:
def get_financial_element_columns():
    return ["type", "name", "unit", "decimals", "value", "context"]

---
# Shares Outstanding

In [25]:
def get_shared_outstanding(soup):
    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:SharesOutstanding")
    if values: 
        return values[0]

    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:CommonStockSharesOutstanding")
    if values: 
        return values[0]

    # assert "No shared outstanding found"
    return -np.inf

In [26]:
SHARES_OUTSTANDING = int(get_shared_outstanding(soup))
SHARES_OUTSTANDING

us-gaap:commonstocksharesoutstanding                                             shares -3    244190000      


244190000

In [27]:
names = re.compile("|".join([
    rf"{NAMESPACE}:SharesOutstanding",
    rf"{NAMESPACE}:CommonStockSharesOutstanding",
    rf"{NAMESPACE}:CommonStockOtherSharesOutstanding",
]).lower())

shares_outstandings = find_financial_elements(soup, names)
df_ShareOutstanding = pd.DataFrame(
    [
        [
            element.name, 
            element['unitref'], 
            int(element['decimals']) if element['decimals'].lower() != 'inf' else np.inf, 
            float(element.text), 
            element['contextref']
        ]
        for element in shares_outstandings
    ],
    columns=["name", "unit", "decimals", "value", "context"]
)
df_ShareOutstanding

,name,unit,decimals,value,context
0,us-gaap:commonstocksharesoutstanding,shares,-3,244190000,i00aae6ee0447424d9c01fc5f8f3bd82d_I20210930


---
# Statements of Income (P/L)

In [28]:
PL = []

In [29]:
from dateutil import parser
parser.parse("2021-05-01")

datetime.datetime(2021, 5, 1, 0, 0)

## Revenues

In [30]:
def get_revenues(soup):
    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:Revenues")
    if values: 
        return values[0]

    # assert "No revenues found"
    return -np.inf

In [31]:
#revenues = find_financial_elements(soup=soup, element_names=re.compile(rf"{NAMESPACE}:Revenues".lower()))
#display_elements(revenues)
#PL += get_records_for_financial_elements(revenues)

names = rf"{NAMESPACE}:Revenues".lower()
PL += get_records_for_financial_element_names(soup=soup, names=names)

REVENUES = get_revenues(soup)
REVENUES

us-gaap:revenues                                                                 usd   -3    1681000000     
us-gaap:revenues                                                                 usd   -3    1681000000     


1681000000.0

## Cost of Revenues

In [32]:
names = re.compile("|".join([
    rf"^{NAMESPACE}:CostOfRevenue$",
    rf"^{NAMESPACE}:CostOfGoods$",
    rf"^{NAMESPACE}:CostOfGoodsAndServicesSold$",
]).lower())
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:costofgoodsandservicessold                                               usd   -3    1356168000     


## ***___# Gross Profit___***

In [33]:
def get_gross_profit(soup):
    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:GrossProfit")
    if values: 
        return values[0]

    # assert "No GrossProfit found"
    return -np.inf

In [34]:
GROSS_PROFIT = get_gross_profit(soup)
GROSS_PROFIT

us-gaap:grossprofit                                                              usd   -3    324832000      


324832000.0

## Operating Expenses

### Research and Development

In [35]:
names = f"{NAMESPACE}:ResearchAndDevelopmentExpense".lower()
PL += get_records_for_financial_element_names(soup, names)

us-gaap:researchanddevelopmentexpense                                            usd   -3    40790000       


### Administrative Expense

In [36]:
names = f"{NAMESPACE}:SellingGeneralAndAdministrativeExpense"
PL += get_records_for_financial_element_names(soup, names)

us-gaap:sellinggeneralandadministrativeexpense                                   usd   -3    72581000       


### Other operating expenses

In [37]:
find_financial_elements(soup=soup, element_names=names)

[<us-gaap:sellinggeneralandadministrativeexpense contextref="icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20210930" decimals="-3" id="id3VybDovL2RvY3MudjEvZG9jOmE2M2JlYTQzZTYzYTQwYTQ5MzU3MzVhNWM1ZDNhNGJhL3NlYzphNjNiZWE0M2U2M2E0MGE0OTM1NzM1YTVjNWQzYTRiYV8xNi9mcmFnOjZiNWVhZGI5NGQ0NTQzMjA5MzZjM2YyZTcwOWNkNjE3L3RhYmxlOjBlNzQ1ZjkzNGMyMjQzMWM5ZDZkNThmMDA4NzI4ZmMwL3RhYmxlcmFuZ2U6MGU3NDVmOTM0YzIyNDMxYzlkNmQ1OGYwMDg3MjhmYzBfNi0xLTEtMS0xNzUw_56509885-f751-439e-a263-25e0e9d168b2" unitref="usd">72581000</us-gaap:sellinggeneralandadministrativeexpense>]

In [38]:
names = f"{NAMESPACE}:OtherCostAndExpenseOperating"
PL += get_records_for_financial_element_names(soup=soup, names=names)

## ***___# Total Operating Expenses___***

In [39]:
names = f"{NAMESPACE}:OperatingExpenses".lower()
get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:operatingexpenses                                                        usd   -3    113371000      


[['debit',
  'us-gaap:operatingexpenses',
  'usd',
  -3,
  113371000.0,
  'icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20210930']]

## ***___# Operating Income___***

$GrossProfit - Total Operating Expenses$

In [40]:
def get_operating_income(soup):
    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:OperatingIncomeLoss")
    if values: 
        return values[0]

    # assert "No OperatingIncomeLoss found"
    return -np.inf

In [41]:
OPERATING_INCOME = get_operating_income(soup)
OPERATING_INCOME

us-gaap:operatingincomeloss                                                      usd   -3    211461000      


211461000.0

## Other Expenses

### Interest Expense

* [Investopedia - What Is an Interest Expense?](https://www.investopedia.com/terms/i/interestexpense.asp)

> An interest expense is the cost incurred by an entity for borrowed funds. Interest expense is a non-operating expense shown on the income statement. It represents interest payable on any borrowings – bonds, loans, convertible debt or lines of credit. It is essentially calculated as the interest rate times the outstanding principal amount of the debt. Interest expense on the income statement represents ***interest accrued during the period*** covered by the financial statements, and **NOT the amount of interest paid over that period**. While interest expense is tax-deductible for companies, in an individual's case, it depends on his or her jurisdiction and also on the loan's purpose.  
>
> For most people, mortgage interest is the single-biggest category of interest expense over their lifetimes as interest can total tens of thousands of dollars over the life of a mortgage as illustrated by online calculators.

In [42]:
names = f"{NAMESPACE}:InterestExpense"
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:interestexpense                                                          usd   -3    12896000       


### Other Non-operating Expenses

In [43]:
names = f"{NAMESPACE}:OtherNonoperatingIncomeExpense"
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:othernonoperatingincomeexpense                                           usd   -3    501000         


## Income Tax

In [44]:
names = f"{NAMESPACE}:IncomeTaxExpenseBenefit"
PL += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:incometaxexpensebenefit                                                  usd   -3    17219000       


## ***___# Net Income___***

$GrossProfit - (Operating Expenses + NonOperating Expense) - Tax$

In [45]:
def get_net_income(soup):
    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:NetIncomeLoss")
    if values: 
        return values[0]

    # assert "No NetIncomeLoss found"
    return -np.inf

In [46]:
names = f"{NAMESPACE}:NetIncomeLoss"
NET_INCOME = get_net_income(soup)
NET_INCOME

us-gaap:netincomeloss                                                            usd   -3    180854000      


180854000.0

## ***___# Net Income Per Share___***

* [US GAAP - Is Net Income Per Share the same with EPS?](https://money.stackexchange.com/questions/148015/us-gaap-is-net-income-per-share-the-same-with-eps)

In [47]:
def get_eps(soup):
    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:EarningsPerShareBasic")
    if values: 
        return values[0]

    values = get_values_for_financial_element_names(soup=soup, names=rf"{NAMESPACE}:EarningsPerShareBasicAndDiluted")
    if values: 
        return values[0]

    return -np.inf

In [48]:
EPS = get_eps(soup)

us-gaap:earningspersharebasic                                                    usdPerShare 2     0.74           


## P/L

Is ```us-gaap:othernonoperatingincomeexpense``` credit or debit? As the value is **negative** and so is in the Income Statement, is shoudl be credit -> To be confirmed. 

In [49]:
df_PL = pd.DataFrame(PL, columns=get_financial_element_columns())
df_PL

,type,name,unit,decimals,value,context
0,credit,us-gaap:revenues,usd,-3,1681000000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...
1,debit,us-gaap:costofgoodsandservicessold,usd,-3,1356168000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...
2,debit,us-gaap:researchanddevelopmentexpense,usd,-3,40790000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...
3,debit,us-gaap:sellinggeneralandadministrativeexpense,usd,-3,72581000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...
4,debit,us-gaap:interestexpense,usd,-3,12896000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...
5,credit,us-gaap:othernonoperatingincomeexpense,usd,-3,501000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...
6,debit,us-gaap:incometaxexpensebenefit,usd,-3,17219000,icb4fe7b89cc949678c0215fb0c6c75ed_D20210701-20...


In [50]:
credits = df_PL[df_PL['type'] == 'credit']['value'].sum()
credits

1681501000.0

In [51]:
debits = df_PL[df_PL['type'] == 'debit']['value'].sum()
debits

1499654000.0

In [52]:
credits - debits  # Equal to the Net Income

181847000.0

---
# Balance Sheet (B/S)

In [53]:
BS = []

## Cash & Cash Equivalents

Look for the cash and cash equivalents for the reporting perid in the Balance Sheet and Cash Flow statements of the  10-K.

In [54]:
names = f"{NAMESPACE}:CashAndCashEquivalentsAtCarryingValue"
BS += get_records_for_financial_element_names(soup=soup, names=names)

us-gaap:cashandcashequivalentsatcarryingvalue                                    usd   -3    545592000      


In [55]:
df_BS = pd.DataFrame(BS, columns=get_financial_element_columns())
df_BS

,type,name,unit,decimals,value,context
0,credit,us-gaap:cashandcashequivalentsatcarryingvalue,usd,-3,545592000,i00aae6ee0447424d9c01fc5f8f3bd82d_I20210930
